In [1]:
import pandas as pd
import boto3
from s3path import S3Path
import glob
import numpy as np

# Import custom modules
%load_ext autoreload
%autoreload 2
#!python -m spacy download en_core_web_lg
import sys
sys.path.append('../')
from normalize import organize, nlp_proc

In [2]:
bucket_path = S3Path('/tab-data-extraction-sandbox/extract_output/rent_rolls/')
DetecTable_paths = list(bucket_path.glob('DetecTable/multifamily/**/*.csv'))
textract_paths = list(bucket_path.glob('textract/**/**/*-1-tables.csv'))

### TODO
1. Iterate over rows of CSV - save index of each row and calculate max similarity to know rent roll categories 
2. Track threshold change in similarity to detect the bounds of the header
3. Validate against labels created by Madi

In [3]:
textract_paths[0]

S3Path('/tab-data-extraction-sandbox/extract_output/rent_rolls/textract/60558_RentRoll.pdf-analysis/60558_rentroll/page-1-tables.csv')

In [4]:
# 1 - iterate over rows of CSVs
# Different call here for textract vs. camelot
# for textract, set header = None
tbl1 = pd.read_csv("s3:/" + str(DetecTable_paths[1]))
tbl2 = pd.read_csv("s3:/" + str(textract_paths[1]), skiprows=1, header=None)

### Compute similarity to known standard

In [5]:
# Read in manual labels, save unique lbx headers as standard
rr_labels = pd.read_csv('s3://tab-data-extraction-sandbox/manual_review/MadiRettew_InternProject_2-19-21_TZreview.csv')
mf_rr_kb = pd.Series(rr_labels.lbx_header.unique())
mf_rr_kb = mf_rr_kb[(mf_rr_kb!='None') & (mf_rr_kb.isna()==False)]
mf_rr_kb.to_csv('../data/rr_multifamily_header.csv', index=False)
mf_rr_kb.head()

1    Tenant Contract Rent
2      Employee Allowance
3               Alarm Fee
4             Unit Number
5         Application Fee
dtype: object

In [7]:
# Test get_max_sim method
df_sim = nlp_proc.header_detector(tbl2, mf_rr_kb).get_max_sim()

../normalize/nlp_proc.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_mat = [[x.similarity(y) for x in to_map_pipe] for y in kb_pipe]


In [11]:
# Test get_header_candidates method
header_id_zscore = nlp_proc.header_detector(tbl2, mf_rr_kb).get_header_candidates()

../normalize/nlp_proc.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_mat = [[x.similarity(y) for x in to_map_pipe] for y in kb_pipe]


In [12]:
# Test get_header_start_end method
header_start_end = nlp_proc.header_detector(tbl2, mf_rr_kb).get_header_start_end()

../normalize/nlp_proc.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_mat = [[x.similarity(y) for x in to_map_pipe] for y in kb_pipe]


In [14]:
nlp_proc.header_detector(tbl2, mf_rr_kb).get_header_start_end()

../normalize/nlp_proc.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_mat = [[x.similarity(y) for x in to_map_pipe] for y in kb_pipe]


[0, 0]

In [78]:
# For page 1 of textract grab zscore >1 with row index
from tqdm import tqdm
zscores = []
for f in tqdm(textract_paths):
    source = f.parents[1].stem
    #source = f
    tbl = pd.read_csv("s3:/" + str(f), skiprows=1, header=None)
    df_sim = header_detector(tbl).get_max_sim()
    # Get sum across each row
    col_sum = df_sim.sum(axis=1)
    mean = np.mean(col_sum)
    std = np.std(col_sum)
    for i,v in enumerate(col_sum):
        z = (v-mean)/std
        if z > 1: 
            zscores.append([source,i,v,z])

  0%|          | 0/139 [00:00<?, ?it/s]../normalize/nlp_proc.py:20: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_mat = [[x.similarity(y) for x in to_map_pipe] for y in kb_pipe]
 69%|██████▉   | 96/139 [01:05<00:33,  1.30it/s]<ipython-input-78-25156b94d4d2>:14: RuntimeWarning: invalid value encountered in double_scalars
  z = (v-mean)/std
100%|██████████| 139/139 [01:35<00:00,  1.45it/s]


In [ ]:
# If headers are non-sequentially detected - see if they equal each other. If not, need to separate into separate tables!!
# Example: 60703_1_rentroll/page-1-tables - 0,1 then 33,34

# Also need to be able to detect total garbage output, such as the following which is purely NAs for the most part
# 74051_1_rentroll

In [160]:
tbl1.head()

,0,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,Month Year = 01/2019,NaN,NaN,NaN,NaN,NaN
1,Unit,Unit Type,Unit Resident,Name,Market,Actual,Resident,Other Move In,Lease,Move Out,Balance
2,NaN,NaN,Sq Ft,NaN,Rent,Rent,Deposit,Deposit,Expiration,NaN,NaN
3,NaN,Current/Notice/Vacant Residents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A102,40021x1A,620.00 VACANT,VACANT,"1,230.00",0.00,0.00,0.00,NaN,NaN,0.00


In [79]:
zscores

[['60558_RentRoll', 0, 8.624169064096506, 4.743043885234741],
 ['60558_RentRoll', 1, 5.489504242454156, 2.6108604139293807],
 ['60659_1_rentroll', 0, 6.528592224161702, 3.9461729839106217],
 ['60663_1_rentroll', 0, 8.622459708501532, 3.842007631260695],
 ['60663_1_rentroll', 1, 4.573649621234261, 1.4119869685726343],
 ['60663_2_rentroll', 0, 7.867721937585951, 3.198608650709549],
 ['60663_2_rentroll', 1, 4.57364962123426, 1.3036435739298788],
 ['60663_3_rentroll', 0, 8.622459708501532, 3.8563895012610314],
 ['60663_3_rentroll', 1, 4.573649621234261, 1.4489960852690347],
 ['60670_1_rentroll', 0, 6.573607069521042, 6.417047807344898],
 ['60701_1_rentroll', 0, 8.821738335819829, 5.707118836830578],
 ['60703_1_rentroll', 0, 7.9670267038144305, 4.41024191051327],
 ['60703_1_rentroll', 1, 4.573649621234261, 1.7980307630457297],
 ['60703_1_rentroll', 33, 6.536796552822949, 3.309255178728249],
 ['60703_1_rentroll', 34, 5.201892394357138, 2.2816501001633367],
 ['60713_10_rentroll', 0, 12.267566